In [1]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.15.0
ipykernel        : 6.25.0
ipywidgets       : 8.0.4
jupyter_client   : 7.4.9
jupyter_core     : 5.3.0
jupyter_server   : 1.23.4
jupyterlab       : 3.6.3
nbclient         : 0.5.13
nbconvert        : 6.5.4
nbformat         : 5.9.2
notebook         : 6.5.4
qtconsole        : 5.4.2
traitlets        : 5.7.1


# Introduction

### What is Yu-Gi-Oh

### How dataset was obtained

### Motivation for data analysis, Problem Statement

Questions We Are Looking To Answer 
- Predict price of card (Linear Regressions) \[attk/level/defence/set\] 
- Predict level based on attack and defence 
- Predict "effect monster" Type \[categorical\] based (Classification Tree) 

Pointers <br>
Make a point on how data is cleaned 
Still do EDA, and see the exceptions 
AKA showcase you know your stuff 
Can split monsters based on can summon normally, or need sacrifice to find. 
 - Can become categorical (bucketing, similar to histograms) 
Show the trend of powercreep over the sets

In [2]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

In [8]:
data_file_location = 'card_data.csv'

yugioh_data = pd.read_csv(data_file_location,index_col=0)
yugioh_data.head(10) # quick look of data

,id,name,type,frameType,desc,race,archetype,card_sets,card_images,card_prices,atk,def,level,attribute,pend_desc,monster_desc,scale,linkval,linkmarkers,banlist_info
0,34541863,"""A"" Cell Breeding Device",Spell Card,spell,"During each of your Standby Phases, put 1 A-Co...",Continuous,Alien,"[{'set_name': 'Force of the Breaker', 'set_cod...","[{'id': 34541863, 'image_url': 'https://images...","[{'cardmarket_price': '0.10', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64163367,"""A"" Cell Incubator",Spell Card,spell,Each time an A-Counter(s) is removed from play...,Continuous,Alien,"[{'set_name': ""Gladiator's Assault"", 'set_code...","[{'id': 64163367, 'image_url': 'https://images...","[{'cardmarket_price': '0.07', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91231901,"""A"" Cell Recombination Device",Spell Card,spell,Target 1 face-up monster on the field; send 1 ...,Quick-Play,Alien,"[{'set_name': 'Invasion: Vengeance', 'set_code...","[{'id': 91231901, 'image_url': 'https://images...","[{'cardmarket_price': '0.10', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73262676,"""A"" Cell Scatter Burst",Spell Card,spell,"Select 1 face-up ""Alien"" monster you control. ...",Quick-Play,Alien,"[{'set_name': 'Strike of Neos', 'set_code': 'S...","[{'id': 73262676, 'image_url': 'https://images...","[{'cardmarket_price': '0.12', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,98319530,"""Infernoble Arms - Almace""",Spell Card,spell,While this card is equipped to a monster: You ...,Equip,Noble Knight,"[{'set_name': 'Duelist Nexus', 'set_code': 'DU...","[{'id': 98319530, 'image_url': 'https://images...","[{'cardmarket_price': '0.00', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,37478723,"""Infernoble Arms - Durendal""",Spell Card,spell,While this card is equipped to a monster: You ...,Equip,Noble Knight,"[{'set_name': '2021 Tin of Ancient Battles', '...","[{'id': 37478723, 'image_url': 'https://images...","[{'cardmarket_price': '0.16', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,64867422,"""Infernoble Arms - Hauteclere""",Spell Card,spell,While this card is equipped to a monster: You ...,Equip,Noble Knight,"[{'set_name': '2021 Tin of Ancient Battles', '...","[{'id': 64867422, 'image_url': 'https://images...","[{'cardmarket_price': '0.02', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,90861137,"""Infernoble Arms - Joyeuse""",Spell Card,spell,While this card is equipped to a monster: You ...,Equip,Noble Knight,"[{'set_name': '2021 Tin of Ancient Battles', '...","[{'id': 90861137, 'image_url': 'https://images...","[{'cardmarket_price': '0.18', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,80845034,"""The Sinful Spoils Hunter Fiend""",Spell Card,spell,"Add 1 ""Diabellestarr"" monster from your Deck o...",Quick-Play,Sinful Spoils,NaN,"[{'id': 80845034, 'image_url': 'https://images...","[{'cardmarket_price': '0.00', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,44256816,1st Movement Solo,Spell Card,spell,If you control no monsters: Special Summon 1 L...,Normal,Melodious,"[{'set_name': '2015 Mega-Tin Mega Pack', 'set_...","[{'id': 44256816, 'image_url': 'https://images...","[{'cardmarket_price': '2.85', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Exploratory Data Analysis [EDA]

In [9]:
# Information about the Variables
yugioh_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12815 entries, 0 to 12814
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            12815 non-null  int64  
 1   name          12815 non-null  object 
 2   type          12815 non-null  object 
 3   frameType     12815 non-null  object 
 4   desc          12815 non-null  object 
 5   race          12814 non-null  object 
 6   archetype     7443 non-null   object 
 7   card_sets     12374 non-null  object 
 8   card_images   12815 non-null  object 
 9   card_prices   12815 non-null  object 
 10  atk           8331 non-null   float64
 11  def           7943 non-null   float64
 12  level         7942 non-null   float64
 13  attribute     8331 non-null   object 
 14  pend_desc     322 non-null    object 
 15  monster_desc  322 non-null    object 
 16  scale         337 non-null    float64
 17  linkval       388 non-null    float64
 18  linkmarkers   388 non-null    o

<b> Listing down observations of dataset: </b>

- 20 columns and 12815 rows.
- 1 variables identified as int64 by default, and it seems is indeed Numeric.
- 5 variables identified as float64 by default, and it seems are indeed Numeric.
- 14 variables identified as object by default, 
    - 4 variables are Strings, which represent card text and should not be used for Categorical values.
        - (name, desc, pend_desc, monster_desc)
    - 1 variables is a JSON object, which represent link data to card images and should not be used for Categorical values.
        - (card_images)
    - 3 variables are JSON objects, which represent Categorical values.
        - (card_sets, linkmarkers, banlist_info)
    - 5 variables are Strings, which represent Categorical values.
        - (type, frameType, race, archetype, attribute)
    - 1 variables is a JSON object, which represent multiple Numeric values.
        - (card_prices)


<b> Cleaning actions to be taken </b> <br>
    - Extract out values from relevant JSON objects
    
None of the variables/features seem to have any missing value (have to check again, carefully).
- card_prices is 


# IDEAS
